# Evaporadores
## Determinación del área de intercambio de calor de un evaporador de efecto simple

### Esquema del evaporador de efecto doble:


<img src="./Imagenes/Esquema_Evaporador_EfectoSimple.jpg" height="300" width="300"/>


In [1]:
%matplotlib inline
import numpy as np
from scipy import optimize
from scipy.integrate import ode
import matplotlib.pyplot as mpl
import matplotlib.patches as patches
import random
import math
import pandas as pd
import IPython.core.display as di
from ipywidgets import interact
import notebook
from IPython.display import clear_output, display, HTML, Image,Math, Latex
from IPython.external import mathjax


Cp_disolucion = 1  # Cp en kcal/kg·ºC


def generador_parametros():  # generación de parámetros del problema de un evaporador de doble efecto

    CaudalAlimento = round(random.uniform(20000., 35000.))  # Caudal másico alimento
    W_SolutoAlimento = round(random.uniform(0.01, 0.05), 3)  # Fracción másica soluto en la alimentación
    W_SolutoProducto = round(random.uniform(0.35, 0.55), 3)  # CFracción másica soluto en la salida del segundo elemento
    Temperatura_Alimento = round(random.uniform(15., 25.), 1)  # Temperatura del alimento
    T_SaturacionVapor = round(random.uniform(110., 125.), 1)  # Temperatura de saturación del vapor de agua
    T_SaturacionCondensador = round(random.uniform(45., 75.), 1)  # Temperatura de saturación condensador
    U = round(random.uniform(2500, 2900.))  # Coeff. Global Efecto
    Teb = round(random.uniform(2, 10.), 1)  # Elevación del punto de ebullición

    return CaudalAlimento, W_SolutoAlimento, W_SolutoProducto, Temperatura_Alimento, T_SaturacionVapor, \
           T_SaturacionCondensador, U, Teb


def Regnault(T):
    value = 606.5 - 0.695 * T  # T en ºC y value en kcal/kg
    return value


def Balances(incognitas, A, Wa_s, Wl_s, Ta, T1, T3, Cp, U, teb):
    L, V, W, T2, Area = incognitas
    CalorT1 = Regnault(T1)
    CalorT2 = Regnault(T2)

    # Balance de materia
    values = [A * Wa_s - L * Wl_s]  # Ec. 1
    values.append(A - (L + V))  # Ec. 2

    # B. Entalpico
    values.append(W * CalorT1 + A * Cp * (Ta - T2) - CalorT2* V)  # Ec. 3
    # Ec. Trans. Calor
    values.append(W * CalorT1 - U * Area * (T1 - T2))  # Ec. 4

    # Ec. Inc. Temp. Ebullición
    values.append(T3 - (T2 - teb))  # Ec. 5

    return values



display(HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');CaudalAlimento,W_SolutoAlimento,W_SolutoProducto,Temperatura_Alimento,T_SaturacionVapor,\
           T_SaturacionCondensador,U, Teb=generador_parametros()

      $('#toggleButton').val('Mostrar código')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Esconder código')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Mostrar código"></form>'''))

display(HTML('<h1 style="color:#000000"><strong>Avanzar para generar datos</strong></h1>'))

In [2]:
display(HTML('<h1 style="color:#000000"><strong>Los datos del sistema son:</strong></h1>'))

CaudalAlimento,W_SolutoAlimento,W_SolutoProducto,Temperatura_Alimento,T_SaturacionVapor,\
           T_SaturacionCondensador,U, Teb=generador_parametros()
    
valores=[np.round(CaudalAlimento),np.round((W_SolutoAlimento*100),1),np.round(Temperatura_Alimento,1),
         np.round((W_SolutoProducto*100),1),np.round(T_SaturacionVapor,1),np.round(T_SaturacionCondensador,1),
          np.round(Teb,1),np.round(U) ]


etiquetas=['A(kg/h)','Soluto_A(%)', 'Temp. A(ºC)','Soluto_L(%)','T1(ºC)','T3 (Saturación disolvente) (ºC)','Teb(ºC)',
          'U(kcal/h·m2·ºC)']

data = dict(zip(etiquetas, valores))

values = pd.DataFrame(data,index=['Valores'], columns=etiquetas)


display(values)


,A(kg/h),Soluto_A(%),Temp. A(ºC),Soluto_L(%),T1(ºC),T3 (Saturación disolvente) (ºC),Teb(ºC),U(kcal/h·m2·ºC)
Valores,33928,2.9,15.4,54.8,117.7,55.7,8.4,2635


In [3]:
display(HTML('<h1 style="color:#000000"><strong>Ecuaciones balance de materia</strong></h1>' ))

display(Math(r'A=L+V'))
display(Math(r'A\cdot X_A^s=L\cdot X_{L}^s'))


display(HTML('<h1 style="color:#000000"><strong>Ecuaciones balance de energía</strong></h1>' ))

display(Math(r'W\cdot \lambda_{\left ( T_1 \right )}+A\cdot C_p^{disolución}\cdot \left ( T_A-T_2 \right )\
=V\cdot \lambda_{\left ( T_2 \right )}'))


display(HTML('<h1 style="color:#000000"><strong>Caudal de calor intercambiado</strong></h1>' ))
display(Math(r'Q=W\cdot \lambda_{\left ( T_1 \right )}=U\cdot Area\cdot \left ( T_1-T_2 \right )'))



display(HTML('<h1 style="color:#000000"><strong>Temperatura de sobrecalentamiento</strong></h1>' ))
display(Math(r'T_3=T_2-T_{teb_1}'))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
display(HTML('<h1 style="color:#000000"><strong>El resultado de las corrientes del sistema y del área del intercambiador es:</strong></h1>' ))

Resultado = optimize.fsolve(Balances, [1000,  1000, 1000, 50,  50],\
                                args=(CaudalAlimento, W_SolutoAlimento ,W_SolutoProducto,Temperatura_Alimento, \
                                T_SaturacionVapor,T_SaturacionCondensador,Cp_disolucion,U, Teb), xtol=1e-06, maxfev=500)


L, V,   W, T2, Area=Resultado


    
valores_resultado=[int(np.round(L)),int(np.round(V)),int(np.round(W)),float(np.round(T2,1)),int(np.round(Area)),\
                  int(np.round(Regnault(T_SaturacionVapor))),int(np.round(Regnault(T2)))]

#print (valores_resultado)

etiquetas_resultado=['L(kg/h)','V (kg/h)','W(kg/h)','T2(ºC)','Area(m2)','Calor latente(T1) (kcal/kg)',\
                     'Calor latente(T2) (kcal/kg)']

data_resultado = dict(zip(etiquetas_resultado,valores_resultado))


values_resultado = pd.DataFrame(data_resultado ,index=['Valores'], columns=etiquetas_resultado)


display(values_resultado)


,L(kg/h),V (kg/h),W(kg/h),T2(ºC),Area(m2),Calor latente(T1) (kcal/kg),Calor latente(T2) (kcal/kg)
Valores,1795,32133,37563,64.1,140,525,562


In [5]:
display(HTML('''

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Programado con Jupyter Notebook en Python 3.6. </footer>'''))